<a href="https://colab.research.google.com/github/SteBaum/Nigeria-Bank/blob/main/Yolov5_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Connect to Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Install cocodataset
!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-cj0bty7s
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-cj0bty7s
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263952 sha256=19deedfd09cacc8ce8a76903b6192eccd9103c0927b69f60aa6592d89350bff0
  Stored in directory: /tmp/pip-ephem-wheel-cache-t53rs1c6/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.4
    Uninstalling pycocotools-2.0.4:
      Successfully uninstalled pycocotools-2.0.4


In [ ]:
# Import modules

from pathlib import Path
from tqdm import tqdm
import numpy as np
import json
import cv2
from IPython.display import display
from sklearn.model_selection import train_test_split
import seaborn as sns
import PIL.Image as Image
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
import ast
import os
import re

In [ ]:
# Load csv dataframe
bankreports = pd.read_csv("/content/gdrive/MyDrive/Liberia_bank/via_project_7Jan2022_14h36m_csv.csv")

In [ ]:
bankreports

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,1-2.jpg,144789,{},2,0,"{""name"":""rect"",""x"":20,""y"":136,""width"":200,""hei...",{}
1,1-2.jpg,144789,{},2,1,"{""name"":""rect"",""x"":276,""y"":39,""width"":201,""hei...",{}
2,1-2-pages.jpg,120719,{},2,0,"{""name"":""rect"",""x"":14,""y"":200,""width"":220,""hei...",{}
3,1-2-pages.jpg,120719,{},2,1,"{""name"":""rect"",""x"":15,""y"":107,""width"":219,""hei...",{}
4,Addiko-bank-st.jpg,187754,{},2,0,"{""name"":""rect"",""x"":36,""y"":377,""width"":393,""hei...",{}
...,...,...,...,...,...,...,...
1586,Yemen-Gulf-bank-statement-template-Word-and-PD...,147935,{},2,1,"{""name"":""rect"",""x"":49,""y"":359,""width"":373,""hei...",{}
1587,Yemen-Gulf-Bank-statement-template-in-Excel-an...,172033,{},2,0,"{""name"":""rect"",""x"":31,""y"":274,""width"":405,""hei...",{}
1588,Yemen-Gulf-Bank-statement-template-in-Excel-an...,172033,{},2,1,"{""name"":""rect"",""x"":31,""y"":405,""width"":404,""hei...",{}
1589,Yemen-International-bank-of-Yemen-proof-of-add...,208654,{},1,0,"{""name"":""rect"",""x"":52,""y"":222,""width"":370,""hei...",{}


In [ ]:
# Check the region_shape_attributes column
bankreports.region_shape_attributes[1]

'{"name":"rect","x":276,"y":39,"width":201,"height":295}'

In [ ]:
# Seperate the region_shape_attributes into different columns
df = pd.json_normalize(bankreports.region_shape_attributes.apply(ast.literal_eval))

In [ ]:
# Join the newly created columns to the dataframe
bankreports = bankreports.join(df)

In [ ]:
bankreports.head()

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes,name,x,y,width,height
0,1-2.jpg,144789,{},2,0,"{""name"":""rect"",""x"":20,""y"":136,""width"":200,""hei...",{},rect,20,136,200,200
1,1-2.jpg,144789,{},2,1,"{""name"":""rect"",""x"":276,""y"":39,""width"":201,""hei...",{},rect,276,39,201,295
2,1-2-pages.jpg,120719,{},2,0,"{""name"":""rect"",""x"":14,""y"":200,""width"":220,""hei...",{},rect,14,200,220,121
3,1-2-pages.jpg,120719,{},2,1,"{""name"":""rect"",""x"":15,""y"":107,""width"":219,""hei...",{},rect,15,107,219,27
4,Addiko-bank-st.jpg,187754,{},2,0,"{""name"":""rect"",""x"":36,""y"":377,""width"":393,""hei...",{},rect,36,377,393,128


In [ ]:
type(bankreports.filename[27])

str

In [ ]:
# Split the images (and not annotations) into train and validation samples, training sample being 90%
train_reports, val_reports = train_test_split(list(bankreports['filename'].drop_duplicates()), test_size=0.1)
len(train_reports), len(val_reports)

(846, 94)

In [ ]:
# Define a function that creaets the dataset with
# the following images and text annotation files.

def create_dataset(reports, dataset_type):
  images_path = Path(f"content/gdrive/MyDrive/Liberia_bank/reports/images/{dataset_type}")
  images_path.mkdir(parents=True, exist_ok=True) # path and folder for image file

  labels_path = Path(f"content/gdrive/MyDrive/Liberia_bank/reports/labels/{dataset_type}")
  labels_path.mkdir(parents=True, exist_ok=True) # path and folder for annoation files

  for item in reports:
    # iterate over rows in the dataframe
    try:    
      abs_path = os.path.join("/content/gdrive/MyDrive/Liberia_bank/bankreport_images", item) #create path
            
  # Open the image files and save them in the corresponding folders
      img = Image.open(abs_path)
      img = img.convert("RGB")
      img.save(str(images_path / item), "JPEG")
      im = cv2.imread(abs_path) # We need this module in order to get the image shape.
      h, w, _ = im.shape # Get the image height and width for the normalization.
      label_name = item.replace('.jpg',".txt" )
      # Open a text file with the annotation coordinates
      with (labels_path / label_name).open("w") as label_file:
        for idx, col in bankreports.iterrows():
          if item == col[0]:
            x = bankreports.iloc[idx]["x"]
            y = bankreports.iloc[idx]["y"]
            width = bankreports.iloc[idx]["width"]
            height = bankreports.iloc[idx]["height"]

          # Write the annotation coordinates into a text file
            label_file.write(f"0 {(x + width / 2)/w} {(y + height / 2)/h} {width/w} {height/h}\n")
            
    except: # accept the error and continue
      continue


In [ ]:
create_dataset(train_reports, 'train')
create_dataset(val_reports, 'val')

In [ ]:
# Go into the wright folder
cd /content/gdrive/MyDrive/Liberia_bank

/content/gdrive/MyDrive/Liberia_bank


In [ ]:
# Install yolov5 and requirements file and import some modules
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
from yolov5 import utils
display = utils.notebook_init()  # checks


YOLOv5 🚀 v6.0-217-gd8b5beb torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.2/166.8 GB disk)


In [ ]:
# Configurating for the clothing data set that you have to change manually for this dataset
!gdown --id 1ZycPS5Ft_0vlfgHnLsfvZPhcH6qOAqBO -O data/clothing.yaml


Downloading...
From: https://drive.google.com/uc?id=1ZycPS5Ft_0vlfgHnLsfvZPhcH6qOAqBO
To: /content/yolov5/data/clothing.yaml
100% 172/172 [00:00<00:00, 345kB/s]


In [ ]:
# Launch training process
!python train.py --img 640 --batch 1 --epochs 30 \
  --data ./data/reports.yaml --cfg ./models/yolov5n.yaml --weights yolov5n.pt \
  --name yolov5n_reports --cache

train: weights=yolov5n.pt, cfg=./models/yolov5n.yaml, data=./data/reports.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=30, batch_size=1, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5n_reports, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-217-gd8b5beb torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7,

In [ ]:
!python detect.py --weights /content/gdrive/MyDrive/Liberia_bank/yolov5/runs/train/yolov5n_reports2/weights/best.pt --img 416 --conf 0.4 --source /content/gdrive/MyDrive/Liberia_bank/reports/images/val

detect: weights=['/content/gdrive/MyDrive/Liberia_bank/yolov5/runs/train/yolov5n_reports2/weights/best.pt'], source=/content/gdrive/MyDrive/Liberia_bank/reports/images/val, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-217-gd8b5beb torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 213 layers, 1760518 parameters, 0 gradients, 4.2 GFLOPs
image 1/94 /content/gdrive/MyDrive/Liberia_bank/reports/images/val/Albania-Bank-of-Albania-bank-statement-template-in-Word-and-PDF-format.jpg: 416x320 1 0, Done. (0.015s)
image 2/94 /content/gdrive/MyDrive/Liberia_bank/reports/images/val/Albania-ProCreditBank-proof-of-ad